# 📋 Multi-Agent Insurance Claim Orchestration

## Overview
The `orchestration.ipynb` notebook implements a **sophisticated multi-agent orchestration system** for insurance claim processing using **Microsoft Semantic Kernel** and **Azure AI Agent Service**. This notebook demonstrates advanced concurrent agent coordination to analyze insurance claims from multiple specialized perspectives simultaneously.

## 🏗️ Architecture & Components

### Core Technologies
- **Microsoft Semantic Kernel**: Agent orchestration framework
- **Azure AI Agent Service**: Cloud-based agent hosting and management
- **Concurrent Orchestration**: Parallel execution of multiple AI agents
- **Azure Cosmos DB Integration**: Real-time data access through custom plugins
- **Azure Identity**: Secure authentication for Azure services

### Agent Specializations
The system creates three specialized AI agents that work concurrently:

1. **🔍 Claim Reviewer Agent**
   - Validates claim documentation completeness
   - Analyzes damage assessments and cost estimates
   - Identifies inconsistencies or missing information
   - Provides VALID/QUESTIONABLE/INVALID determinations
   - Equipped with Cosmos DB plugin for data retrieval

2. **⚠️ Risk Analyzer Agent**
   - Detects fraud patterns and suspicious indicators
   - Assesses claim authenticity and credibility
   - Analyzes timing, amounts, and circumstances
   - Provides LOW/MEDIUM/HIGH risk assessments
   - Equipped with Cosmos DB plugin for historical analysis

3. **📋 Policy Checker Agent**
   - Validates coverage against policy terms
   - Interprets limits, deductibles, and exclusions
   - Handles multiple policy types (Auto, Commercial, Motorcycle, etc.)
   - Provides COVERED/NOT COVERED/PARTIAL COVERAGE determinations
   - Uses Azure AI Search for policy document analysis

## 🔧 Key Functions

### `create_specialized_agents()`
- Establishes Azure AI Agent client connections
- Configures each agent with specialized instructions and capabilities
- Sets up Cosmos DB plugins for data-enabled agents
- Configures polling options for optimal performance
- Returns a collection of ready-to-use specialized agents

### `run_insurance_claim_orchestration()`
- **Concurrent Processing**: All three agents analyze claims simultaneously
- **Intelligent Task Distribution**: Each agent receives specialized instructions
- **Real-time Data Access**: Agents can retrieve claim data using provided claim IDs
- **Comprehensive Reporting**: Generates unified analysis reports combining all agent outputs
- **Error Handling**: Robust exception management and resource cleanup
- **Progress Tracking**: Detailed logging of orchestration stages

## 🚀 Orchestration Flow

1. **Agent Creation**: Initializes three specialized insurance processing agents
2. **Concurrent Orchestration Setup**: Creates parallel execution framework
3. **Task Distribution**: Assigns specialized analysis tasks to each agent
4. **Parallel Execution**: All agents work simultaneously on their respective analyses
5. **Result Aggregation**: Collects and consolidates outputs from all agents
6. **Report Generation**: Creates comprehensive analysis report with all findings
7. **Resource Cleanup**: Properly terminates runtime and releases resources

## 📊 Output Format

The system generates a comprehensive **Insurance Claim Analysis Report** that includes:

- **Individual Agent Assessments**: Detailed analysis from each specialized perspective
- **Claim Validity Status**: Complete documentation and accuracy assessment
- **Risk Profile Analysis**: Fraud indicators and authenticity evaluation
- **Policy Coverage Determination**: Coverage eligibility and terms validation
- **Unified Recommendations**: Consolidated next steps based on all agent analyses

## 💡 Advanced Features

- **Concurrent Execution**: True parallelism for faster processing
- **Database Integration**: Real-time access to claim and policy data
- **Flexible Configuration**: Environment-based model and endpoint configuration
- **Comprehensive Logging**: Detailed progress tracking and status updates
- **Timeout Management**: 5-minute timeout for orchestration completion
- **Error Recovery**: Graceful handling of individual agent failures

## 🎯 Use Case Example

The notebook includes a practical example demonstrating the orchestration of claim analysis for "CL002", showing how the system processes claim details and retrieves additional data to provide comprehensive multi-agent analysis.

This implementation represents a cutting-edge approach to insurance claim processing, leveraging the power of concurrent AI agents to provide thorough, multi-perspective analysis that would traditionally require multiple human experts working in sequence.


In [1]:
# Import necessary libraries for Semantic Kernel orchestration
import asyncio
import os
from typing import Dict, Any
from datetime import timedelta
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import (
    AzureAIAgent, 
    ConcurrentOrchestration
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions
from azure.ai.agents.models import AzureAISearchQueryType, AzureAISearchTool, ListSortOrder, MessageRole
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from azure.identity import AzureCliCredential  # async credential

# Import the Cosmos DB plugin
from agents.tools import CosmosDBPlugin
from dotenv import load_dotenv

load_dotenv(override=True)  # This forces a reload of the .env file


async def create_specialized_agents():
    """Create our specialized insurance processing agents using Semantic Kernel."""
    
    print("🔧 Creating specialized insurance agents...")
    
    # Create Cosmos DB plugin instances for different agents
    cosmos_plugin_claims = CosmosDBPlugin()
    cosmos_plugin_risk = CosmosDBPlugin()
    
    # Get environment variables
    endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
    model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4.1-mini")
    
    agents = {}
    
    async with DefaultAzureCredential() as creds:
        client = AzureAIAgent.create_client(credential=creds, endpoint=endpoint)
        
        # Create Claim Reviewer Agent with Cosmos DB access
        print("🔍 Creating Claim Reviewer Agent...")
        claim_reviewer_definition = await client.agents.create_agent(
            model=model_deployment,
            name="ClaimReviewer",
            description="Expert Insurance Claim Reviewer Agent specialized in analyzing and validating insurance claims",
            instructions="""You are an expert Insurance Claim Reviewer Agent specialized in analyzing and validating insurance claims. 
            Your primary responsibilities include:
            1. Use the Cosmos DB plugin to retrieve claim data by claim_id, then:
            2.Review all claim details (dates, amounts, descriptions).
            3. Verify completeness of documentation and supporting evidence.
            4. Analyze damage assessments and cost estimates for reasonableness.
            5. Validate claim details against policy requirements.
            6. Identify inconsistencies, missing info, or red flags.
            7. Provide a detailed assessment with specific recommendations.

            **Response Format**:

            A short paragraph description if the CLAIM STATUS is: VALID / QUESTIONABLE / INVALID ; Analysis: Summary of findings by component; Any missing Info / Concerns: List of issues or gaps;
            Next Steps: Clear, actionable recommendations
    """
        )
        
        claim_reviewer_agent = AzureAIAgent(
            client=client,
            definition=claim_reviewer_definition,
            plugins=[cosmos_plugin_claims]
        )
        
        # Create Risk Analyzer Agent with Cosmos DB access
        print("⚠️ Creating Risk Analyzer Agent...")
        risk_analyzer_definition = await client.agents.create_agent(
            model=model_deployment,
            name="RiskAnalyzer",
            instructions="""You are the Risk Analysis Agent. Your role is to evaluate the authenticity of insurance claims and detect potential fraud using available claim data.
            Core Functions:
            - Analyze historical and current claim data
            - Identify suspicious patterns, inconsistencies, or anomalies
            - Detect fraud indicators
            - Assess claim credibility and assign a risk score
            - Recommend follow-up actions if warranted

            Assessment Guidelines:
            - Use the Cosmos DB plugin to access claim records
            - Look for unusual timing, inconsistent descriptions, irregular amounts, or clustering
            - Check for repeat claim behavior or geographic overlaps
            - Assess the overall risk profile of each claim

            Fraud Indicators to Watch For:
            - Claims with irregular timing
            - Contradictory or vague damage descriptions
            - Unusual or repetitive claim amounts
            - Multiple recent claims under same or related profiles
            - Geographic or temporal clustering of incidents

            Output Format:
            - Risk Level: LOW / MEDIUM / HIGH
            - Risk Analysis: Brief summary of findings
            - Indicators: List of specific fraud signals (if any)
            - Risk Score: 1–10 scale
            - Recommendation: Investigate / Monitor / No action needed

            Base all assessments strictly on the available claim data. Use structured reasoning and avoid assumptions beyond the data.
            """,
        )
        
        risk_analyzer_agent = AzureAIAgent(
            client=client,
            definition=risk_analyzer_definition,
            plugins=[cosmos_plugin_risk]
        )
        
        ai_agent_settings = AzureAIAgentSettings(model_deployment_name= os.environ.get("MODEL_DEPLOYMENT_NAME"), azure_ai_search_connection_id=os.environ.get("AZURE_AI_AGENT_ENDPOINT"))        
        ai_search = AzureAISearchTool(
            index_connection_id=os.environ.get("AZURE_AI_CONNECTION_ID"), 
            index_name="insurance-documents-index"
        )

        # Create agent definition
        policy_agent_definition = await client.agents.create_agent(
            name="PolicyChecker", 
            model=os.environ.get("MODEL_DEPLOYMENT_NAME"),
            instructions=""""
            You are the Policy Checker Agent.

            Your task is to summarize a policy based on policy number.

            Instructions:
            - Do not analyze claim details directly.
            - Use your search tool to locate policy documents by policy number or policy type.
            - Identify relevant exclusions, limits, and deductibles.
            - Base your determination only on the contents of the retrieved policy.

            Output Format:
            - Policy Number: [Policy number]
            - Main important details
            - Reference and quote specific policy sections that support your determination.
            - Clearly explain how the policy language leads to your conclusion.

            Be precise, objective, and rely solely on the policy content.
            """,
            tools=ai_search.definitions,
            tool_resources=ai_search.resources,
            headers={"x-ms-enable-preview": "true"},
        )

        policy_checker_agent = AzureAIAgent(
            client=client, 
            definition=policy_agent_definition
        )

        agents = {
            'claim_reviewer': claim_reviewer_agent,
            'risk_analyzer': risk_analyzer_agent,
            'policy_checker': policy_checker_agent
        }
        
        print("✅ All specialized agents created/loaded successfully!")
        return agents, client

async def run_insurance_claim_orchestration(claim_id: str, policy_number: str):
    """Orchestrate multiple agents to process an insurance claim concurrently using only the claim ID."""
    
    print(f"🚀 Starting Concurrent Insurance Claim Processing Orchestration")
    print(f"{'='*80}")
    
    # Create our specialized agents
    agents, client = await create_specialized_agents()
    
    # Create concurrent orchestration with all three agents
    orchestration = ConcurrentOrchestration(
        members=[agents['claim_reviewer'], agents['risk_analyzer'], agents['policy_checker']]
    )
    
    # Create and start runtime
    runtime = InProcessRuntime()
    runtime.start()
    
    try:        
        # Create task that instructs agents to retrieve claim details first
        task = f"""Analyze the insurance claim with ID: {claim_id} or the policy number {policy_number} and come back with a critical solution for if the credit should be approved.

CRITICAL: ALL AGENTS MUST USE THEIR AVAILABLE TOOLS TO RETRIEVE INFORMATION

AGENT-SPECIFIC INSTRUCTIONS:

Claim Reviewer Agent: 
- MUST USE: get_document_by_claim_id("{claim_id}") to retrieve claim details
- Review all claim documentation and assess completeness
- Validate damage estimates and repair costs against retrieved data
- Check for proper evidence and documentation in the claim data
- Cross-reference claim amounts with industry standards
- Provide VALID/QUESTIONABLE/INVALID determination with detailed reasoning

Risk Analyzer Agent:
- MUST USE: get_document_by_claim_id("{claim_id}") to retrieve claim data
- Analyze the retrieved data for fraud indicators and suspicious patterns
- Assess claim authenticity and credibility based on actual claim details
- Check for unusual timing, amounts, or circumstances in the data
- Look for inconsistencies between different parts of the claim
- Provide LOW/MEDIUM/HIGH risk assessment with specific evidence

Policy Checker Agent (policy_checker_agent):
- YOU DO NOT NEED TO LOOK INTO CLAIMS!
- MUST USE: Your search capabilities to find relevant policy documents by policy number ("{policy_number}") or type found in the claim data
- Search for policy documents using policy numbers
- Identify relevant exclusions, limits, or deductibles from actual policy documents
- Provide COVERED/NOT COVERED/PARTIAL COVERAGE determination with policy references
- Quote specific policy sections that support your determination

IMPORTANT: Each agent MUST actively use their tools to retrieve and analyze actual data. 
Do not provide generic responses - base your analysis on the specific claim data and policy documents retrieved through your tools.
"""
        # Invoke concurrent orchestration
        orchestration_result = await orchestration.invoke(
            task=task,
            runtime=runtime
        )
        
        # Get results from all agents
        results = await orchestration_result.get(timeout=300)  # 5 minute timeout
        
        print(f"\n🎉 All agents completed their analysis!")
        print(f"{'─'*60}")
        
        # Display individual results
        for i, result in enumerate(results, 1):
            agent_name = result.name if hasattr(result, 'name') else f"Agent {i}"
            content = str(result.content)
            print(f"\n🤖 {agent_name} Analysis:")
            print(f"{'─'*40}")
            print(content)
        
        # Create comprehensive analysis report
        comprehensive_analysis = f"""

{chr(10).join([f"### {result.name} Assessment:{chr(10)}{chr(10)}{result.content}{chr(10)}" for result in results])}

"""
        
        print(f"\n✅ Concurrent Insurance Claim Orchestration Complete!")
        return comprehensive_analysis
        
    except Exception as e:
        print(f"❌ Error during orchestration: {str(e)}")
        raise
        
    finally:
        await runtime.stop_when_idle()
        print(f"\n🧹 Orchestration cleanup complete.")

In [2]:
claim_id = "CL001"
policy_number = "LIAB-AUTO-001"  # User provides the specific policy number to search
result = await run_insurance_claim_orchestration(claim_id, policy_number)

🚀 Starting Concurrent Insurance Claim Processing Orchestration
🔧 Creating specialized insurance agents...
🔍 Creating Claim Reviewer Agent...
⚠️ Creating Risk Analyzer Agent...
✅ All specialized agents created/loaded successfully!

🎉 All agents completed their analysis!
────────────────────────────────────────────────────────────

🤖 PolicyChecker Analysis:
────────────────────────────────────────
Policy Number: LIAB-AUTO-001

Main important details:
- This policy is a "Liability Only Auto Insurance" policy that provides the minimum required coverage for legally driving in most states.
- It covers damages and injuries caused to others by the insured but does not cover damage to the insured's own vehicle or injuries to the insured.
- Covered perils include bodily injury liability such as medical expenses, lost wages, pain and suffering compensation, rehabilitation costs for injured third parties, legal defense costs for covered claims, and court-ordered judgments and settlements.

Support